
---


**Projet Dapyness - Estimation de ventes e-commerce** 


---

*L’objectif de ce projet est d’estimer l’évolution du volume de vente de produits vendus en ligne par un site e-commerce en utilisant les données de ces ventes.*

*Pour cela, nous étudions une table de données que nous nommons **df** lors de l'importation.*



*   Nous commençons notre étude par importer, analyser et nettoyer notre jeu de données. 
*   Ensuite, nous créons de nouvelles variables jugées utiles pour la suite de notre étude.
*   Nous analysons ensuite graphiquement nos données.
*   Et enfin, nous procédons à la modélisation de nos données pour espérer obtenir de bonnes prévisions de ventes.






L'ensemble des fichiers codes de ce projet sont décomposés en 5 parties :



*   **01 - Analyse exploratoire 1/2 - Etude des différentes colonnes**
*   02 - Analyse exploratoire 2/2 - Etude des valeurs manquantes
*   03 - création de nouvelles colonnes
*   04 - Etude graphique des variables et tests statistiques
*   05 - Modélisation - Time Series



# **Préparation du Dataset et premières informations**

In [1]:
# Cellule d'imporation des packages nécessaires à l'ensemble des fichiers codes de ce projet

#packages basiques
import pandas as pd
import numpy as np

#partie graphique
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from bokeh.plotting import figure, output_notebook, show
output_notebook()

#tests statistiques
import statsmodels.api 
from scipy.stats import pearsonr

#modélisation
#régression
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error

#modèles arima et sarima
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
# Lecture du fichier .csv correspondant à notre dataset et le stocker dans notre dataframe df
#df = pd.read_csv('data.csv',encoding = "ISO-8859-1")

In [2]:
#Importation du fichier csv chez Sanae 
df=pd.read_csv('data-estimation.csv', sep=';')

In [3]:
#Affichage des premières lignes de df
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom


In [4]:
#Informations sur notre dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
''' Nous avons 541909 lignes dans notre dataset, 8 colonnes, de type object, float ou int
Il semble également y avoir des valeurs manquantes dans les colonnes 'Description' et 'CustomerID' '''

In [5]:
# description des variables quantitatives
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
""" 
--> La colonne Quantity contient des valeurs très élevés et très basses,
 50% des valeurs sont comprises entre 1 et 10
--> La colonne UnitPrice contient également des valeurs très élevés et très basses
50% des valeurs sont comprises entre 1.25 et 4.13
Remarque : Nous ne comptons pas CustomerID comme étant une variable quantitative 
           car représente un identifiant client
"""

# **Etude et Gestion des doublons**

In [ ]:
# Détection des doublons
df.duplicated().sum()
# 5268 doublons

5268

In [6]:
df[df.duplicated()]
# --> ne semblent pas correspondre à de réels doublons

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/01/2010 11:45,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/01/2010 11:45,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,12/01/2010 11:45,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/01/2010 11:45,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,12/01/2010 11:49,2.95,17920.0,United Kingdom
...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,12/09/2011 11:34,0.39,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,12/09/2011 11:34,2.49,14446.0,United Kingdom
541692,581538,22992,REVOLVER WOODEN RULER,1,12/09/2011 11:34,1.95,14446.0,United Kingdom
541699,581538,22694,WICKER STAR,1,12/09/2011 11:34,2.10,14446.0,United Kingdom


In [ ]:
#affichage de tous les articles de la commande '536412
pd.set_option('display.max_rows', None)
df[df.InvoiceNo.isin(['536412'])]

In [ ]:
"""
 Nous emmetons l'hyptohèse d'absence de doublons. 
En effet, la plupart des lignes semblent ne pas réelment correspondre à des doublons.
De plus, un article peut très bien être entré plusieurs fois dans un panier,
ce qui peut expliquer les quelques doublons.
--> Nous ne supprimons pas ces lignes

"""

# **Etude approfondie des différentes colonnes**

In [ ]:
# calcul du nombre de valeurs uniques pour chaque colonnes
print('nombre de factures différentes : ', np.count_nonzero(df['InvoiceNo'].unique()))
print('nombre de code de stock différentes : ', np.count_nonzero(df['StockCode'].unique()))
print('nombre de descriptions différentes : ', np.count_nonzero(df['Description'].unique()))
print('nombre de valeurs de quantités différentes : ', np.count_nonzero(df['Quantity'].unique()))
print('nombre de valeurs de prix différentes : ', np.count_nonzero(df['UnitPrice'].unique()))
print('nombre de clients différents : ', np.count_nonzero(df['CustomerID'].unique()))
print('nombre de pays différents : ', np.count_nonzero(df['Country'].unique()))

nombre de factures différentes :  25900
nombre de code de stock différentes :  4070
nombre de descriptions différentes :  4224
nombre de valeurs de quantités différentes :  722
nombre de valeurs de prix différentes :  1629
nombre de clients différents :  4373
nombre de pays différents :  38


---
**Etude de la colonne InvoiceNo**


---

In [ ]:
# nous remarquons que les factures peuvent contenir des lettres en premiere position
# affichage des factures qui contiennent lettre C
df[df['InvoiceNo'].str.contains("C")].head()
#--> correspondent à des retours

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/01/2010 09:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/01/2010 09:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/01/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/01/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/01/2010 10:24,0.29,17548.0,United Kingdom


In [ ]:
#création d'une table avec uniquement les données pour lesquels la facture commence par la lettre C
data_invoice_C=df[df['InvoiceNo'].str.contains("C")]
# a-t-on des quantités positives dans ce cas ? 
data_invoice_C[data_invoice_C['Quantity'] >0]
# ==> non 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [ ]:
#affichage des factures qui contiennent lettre A
df[df['InvoiceNo'].str.contains("A")]
#--> correspondent à des ajustements de dettes 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,08/12/2011 14:50,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,08/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,08/12/2011 14:52,-11062.06,NaN,United Kingdom


In [ ]:
# essai avec toutes les lettres
df[df['InvoiceNo'].str.contains("Z")]
# --> Nous avons uniquement des factures composées de la lettre 'A' ou 'C' ou sans lettres

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


---
**Etude de la colonne UnitPrice**


---

In [ ]:
# Combien as-t-on de prix = 0 ?
df[df['UnitPrice']==0].UnitPrice.count()

2515

In [ ]:
# Combien as-t-on de prix < 0 ?
print('nombre de prix négatifs : ', df[df['UnitPrice']<0].UnitPrice.count())
df[df['UnitPrice'] < 0]
# --> 2 prix négatifs --> lignes correspondant à des ajustements de dettes

nombre de prix négatifs :  2


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,08/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,08/12/2011 14:52,-11062.06,NaN,United Kingdom


In [ ]:
#ordre décroissant du prix
df.sort_values(by='UnitPrice', ascending=False).head(10)
#--> Nous retrouvons des prix très élevés.
## nous remarquons des données innatendues dans 'Description' et 'StockCode'
# --> Pour ces prix, le stockcode ainsi que la description ne 
# correspondent pas à des articles mais plutôt des autres opérations
# nous verrons cela en détails dans la prochaine partie sur l'étude de la colonne StockCode

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222681,C556445,M,Manual,-1,06/10/2011 15:31,38970.00,15098.0,United Kingdom
524602,C580605,AMAZONFEE,AMAZON FEE,-1,12/05/2011 11:36,17836.46,NaN,United Kingdom
43702,C540117,AMAZONFEE,AMAZON FEE,-1,01/05/2011 09:55,16888.02,NaN,United Kingdom
43703,C540118,AMAZONFEE,AMAZON FEE,-1,01/05/2011 09:57,16453.71,NaN,United Kingdom
15017,537632,AMAZONFEE,AMAZON FEE,1,12/07/2010 15:08,13541.33,NaN,United Kingdom
16356,C537651,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:49,13541.33,NaN,United Kingdom
15016,C537630,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:04,13541.33,NaN,United Kingdom
16232,C537644,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:34,13474.79,NaN,United Kingdom
524601,C580604,AMAZONFEE,AMAZON FEE,-1,12/05/2011 11:35,11586.50,NaN,United Kingdom
299982,A563185,B,Adjust bad debt,1,08/12/2011 14:50,11062.06,NaN,United Kingdom


---
**Etude de la colonne StockCode**


---

In [ ]:
#affichage des différents stockcode lorsque le prix > 100 (--> pour détecter les anomalies)
df[df['UnitPrice'] > 100].StockCode.unique()

array(['22827', 'DOT', '22655', 'M', 'D', 'AMAZONFEE', '22828', '22826',
       '22656', 'BANK CHARGES', '22823', 'C2', 'S', 'POST', '22502', 'B',
       'CRUK'], dtype=object)

In [ ]:
#essai avec toutes les lettres de l'alphabet pour détecter des codes anormaux
df[df['StockCode'].str.contains("Z")].StockCode.unique()

array(['AMAZONFEE', '90214Z'], dtype=object)

In [ ]:
#affichage des lignes pour lesquelles stockcode == C2
print('nombre de prix différents dans le cas de frais de transport : ', df[df['StockCode'] == 'C2'].UnitPrice.unique())
df[df['StockCode'] == 'C2'].head(5)
# --> frais de transport 

nombre de prix différents dans le cas de frais de transport :  [ 50. 150.  15.  18.  25.   0.]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1423,536540,C2,CARRIAGE,1,12/01/2010 14:05,50.0,14911.0,EIRE
12119,537368,C2,CARRIAGE,1,12/06/2010 12:40,50.0,14911.0,EIRE
12452,537378,C2,CARRIAGE,1,12/06/2010 13:06,50.0,14911.0,EIRE
19975,537963,C2,CARRIAGE,1,12/09/2010 11:30,50.0,13369.0,United Kingdom
20016,538002,C2,CARRIAGE,1,12/09/2010 11:48,50.0,14932.0,Channel Islands


In [ ]:
#affichage des lignes pour lesquelles stockcode == D
df[df['StockCode'] == 'D'].head(5)
# --> correspondent à des remises

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/01/2010 09:41,27.50,14527.0,United Kingdom
9038,C537164,D,Discount,-1,12/05/2010 13:21,29.29,14527.0,United Kingdom
14498,C537597,D,Discount,-1,12/07/2010 12:34,281.00,15498.0,United Kingdom
19392,C537857,D,Discount,-1,12/08/2010 16:00,267.12,17340.0,United Kingdom
31134,C538897,D,Discount,-1,12/15/2010 9:14,5.76,16422.0,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == S
df[df['StockCode'] == 'S'].head()
#--> correspondent à des échantillon

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14436,C537581,S,SAMPLES,-1,12/07/2010 12:03,12.95,NaN,United Kingdom
14437,C537581,S,SAMPLES,-1,12/07/2010 12:03,52.00,NaN,United Kingdom
96680,C544580,S,SAMPLES,-1,2/21/2011 14:25,5.74,NaN,United Kingdom
96681,C544580,S,SAMPLES,-1,2/21/2011 14:25,11.08,NaN,United Kingdom
96682,C544580,S,SAMPLES,-1,2/21/2011 14:25,5.79,NaN,United Kingdom


In [ ]:
df[df['StockCode'] == 'S'].CustomerID.unique()
#--> on a uniquement des NaN pour customerID

array([nan])

In [ ]:
#affichage des lignes pour lesquelles stockcode == B
df[df['StockCode'] == 'B']
#--> ajustement dettes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,08/12/2011 14:50,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,08/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,08/12/2011 14:52,-11062.06,NaN,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == M
df[df['StockCode'] == 'M'].head()
# --> correspondent à des commandse entrées manuellement 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,12/01/2010 15:35,1.25,16274.0,United Kingdom
2250,536569,M,Manual,1,12/01/2010 15:35,18.95,16274.0,United Kingdom
5684,536865,M,Manual,1,12/03/2010 11:28,2.55,NaN,United Kingdom
6798,536981,M,Manual,2,12/03/2010 14:26,0.85,14723.0,United Kingdom
7976,537077,M,Manual,12,12/05/2010 11:59,0.42,17062.0,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == 'BANK CHARGES'
df[df['StockCode'] == 'BANK CHARGES'].head()
# --> correspondent à des frais de banque 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4406,536779,BANK CHARGES,Bank Charges,1,12/02/2010 15:08,15.00,15823.0,United Kingdom
14435,C537572,BANK CHARGES,Bank Charges,-1,12/07/2010 12:00,95.38,NaN,United Kingdom
28992,C538680,BANK CHARGES,Bank Charges,-1,12/13/2010 17:10,966.92,NaN,United Kingdom
62508,541505,BANK CHARGES,Bank Charges,1,1/18/2011 15:58,15.00,15939.0,United Kingdom
64573,C541653,BANK CHARGES,Bank Charges,-1,1/20/2011 11:50,1050.15,NaN,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == 'AMAZON FEE'
df[df['Description'] == 'AMAZON FEE'].head()
# on a que des Nan pour customerID

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14514,C537600,AMAZONFEE,AMAZON FEE,-1,12/07/2010 12:41,1.00,NaN,United Kingdom
15016,C537630,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:04,13541.33,NaN,United Kingdom
15017,537632,AMAZONFEE,AMAZON FEE,1,12/07/2010 15:08,13541.33,NaN,United Kingdom
16232,C537644,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:34,13474.79,NaN,United Kingdom
16313,C537647,AMAZONFEE,AMAZON FEE,-1,12/07/2010 15:41,5519.25,NaN,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == 'POST'
df[df['StockCode'] == 'POST'].head()
#--> correspondent à des frais de postage

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,12/01/2010 08:45,18.0,12583.0,France
386,536403,POST,POSTAGE,1,12/01/2010 11:27,15.0,12791.0,Netherlands
1123,536527,POST,POSTAGE,1,12/01/2010 13:04,18.0,12662.0,Germany
5073,536840,POST,POSTAGE,1,12/02/2010 18:27,18.0,12738.0,Germany
5258,536852,POST,POSTAGE,1,12/03/2010 09:51,18.0,12686.0,France


In [ ]:
#affichage des lignes pour lesquelles stockcode == 'DOT'
df[df['StockCode'] == 'DOT'].head()
# --> également des frais de postage
# --> ne concerne qu'un client ou clients non identifiés

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1814,536544,DOT,DOTCOM POSTAGE,1,12/01/2010 14:32,569.77,NaN,United Kingdom
3041,536592,DOT,DOTCOM POSTAGE,1,12/01/2010 17:06,607.49,NaN,United Kingdom
5450,536862,DOT,DOTCOM POSTAGE,1,12/03/2010 11:13,254.43,NaN,United Kingdom
5545,536864,DOT,DOTCOM POSTAGE,1,12/03/2010 11:27,121.06,NaN,United Kingdom
5685,536865,DOT,DOTCOM POSTAGE,1,12/03/2010 11:28,498.47,NaN,United Kingdom


In [ ]:
#affichage des lignes pour lesquelles stockcode == CRUK
df[df['StockCode'] == 'CRUK'].head()
#--> commission, ne concerne qu'un client

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
317508,C564763,CRUK,CRUK Commission,-1,8/30/2011 10:49,1.60,14096.0,United Kingdom
324023,C565382,CRUK,CRUK Commission,-1,09/02/2011 15:45,13.01,14096.0,United Kingdom
333779,C566216,CRUK,CRUK Commission,-1,09/09/2011 15:17,15.96,14096.0,United Kingdom
338848,C566565,CRUK,CRUK Commission,-1,9/13/2011 12:32,52.24,14096.0,United Kingdom
351003,C567655,CRUK,CRUK Commission,-1,9/21/2011 14:40,608.66,14096.0,United Kingdom


---
**Etude de la colonne Country**


---

In [ ]:
#affichage des différents pays présents dans le dataset
df['Country'].unique()
#38 pays différents

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
# Part d'articles différents vendus par pays
df['Country'].value_counts(normalize = True)
# --> 91,4% des articles vendus à UK
#le reste des pays comptabilisent moins de 2% des commandes
# --> données non équilibrées

United Kingdom          0.914320
Germany                 0.017521
France                  0.015790
EIRE                    0.015124
Spain                   0.004674
Netherlands             0.004375
Belgium                 0.003818
Switzerland             0.003694
Portugal                0.002803
Australia               0.002323
Norway                  0.002004
Italy                   0.001482
Channel Islands         0.001399
Finland                 0.001283
Cyprus                  0.001148
Sweden                  0.000853
Unspecified             0.000823
Austria                 0.000740
Denmark                 0.000718
Japan                   0.000661
Poland                  0.000629
Israel                  0.000548
USA                     0.000537
Hong Kong               0.000531
Singapore               0.000423
Iceland                 0.000336
Canada                  0.000279
Greece                  0.000269
Malta                   0.000234
United Arab Emirates    0.000125
European C

---
**Etude de la colonne Quantity**


---

In [ ]:
# calcul du nombre de fois où l'on a des quantités négatives/postives/nulles
print('Nombre de quantités négatives : ', df[df['Quantity']<0].Quantity.count())
print('Nombre de quantités positives : ', df[df['Quantity']>0].Quantity.count())
print('Nombre de quantités nulles : ', df[df['Quantity']==0].Quantity.count())

Nombre de quantités négatives :  10624
Nombre de quantités positives :  531285
Nombre de quantités nulles :  0


In [ ]:
#ordre décroissant de la quantité
df.sort_values(by='Quantity', ascending=False).head()
# --> Nous avons des quantités très élevées et très basses en négatif
# mais cela peut s'expliquer par le fait que la majorité des clients soient des grossistes
# valeurs non abérrantes

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,12/09/2011 09:15,2.08,16446.0,United Kingdom
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,11/25/2011 15:57,0.00,13256.0,United Kingdom
74614,542504,37413,NaN,5568,1/28/2011 12:03,0.00,NaN,United Kingdom
421632,573008,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,4800,10/27/2011 12:26,0.21,12901.0,United Kingdom


In [ ]:
''' En résumé : 

--> Nous avons 3 types de factures différentes : 
    - Les factures 'A' --> ajustements de dettes
    - les factures 'C' --> principalement des annulations de commandes (peut contenir d'autres types de 
    lignes comme des frais) 
    - les factures non lettrées --> principalement des ventes (peut contenir d'autres types de lignes 
    commes frais, transport, frais bancaire...)

--> Nous avons des stockcode qui ne correspondent pas à des produits mais font références à d'autres "opérations" 
comme les frais, transport, postage...
    ==> Dans la suite, nous créerons une nouvelle colonne 'InvoiceType' dans laquelle nous catégoriserons les lignes 
    selon leur type : vente, annulations, dettes, frais, échantillons, ... (voir section création de nouvelles colonnes)

--> Des quantités très élevées et très basses --> valeurs non abérrantes car présence de grossistes 
dans notre base de données. 

--> Des prix très élevés --> lignes non correspondant à des articles mais plutôt des 'opérations' (frais, postage, etc...)
--> 2 prix négatifs --> ajustement de dettes 
--> Des prix nuls --> (étudiés dans la section 'Etude et gestion des valeurs manquantes') 
    correspondent soit à des problèmes de commandes soit des 'cadeaux' ou 'offres'

--> 38 pays différents dont un surreprésenté : UK    

--> Les colonnes 'Description' et 'CustomerID' sont étudiées dans la section suivante 
(Etude et gestion des valeurs manquantes) '''